In [21]:
import helper


ModuleNotFoundError: No module named 'helper'

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/lab-pep-itba/Clase-3---Clasificadores-Bayesianos/master/data/alturas-pesos-mils-train.csv", index_col="Unnamed: 0")
df_test = pd.read_csv("https://raw.githubusercontent.com/lab-pep-itba/Clase-3---Clasificadores-Bayesianos/master/data/alturas-pesos-mils-test.csv", index_col="Unnamed: 0")

In [3]:
df.head()

,Genero,Peso,Altura
7620,Mujer,61.235,162.402
1837,Hombre,97.432,181.908
3311,Hombre,73.324,172.459
9478,Mujer,55.193,157.748
7805,Mujer,56.886,151.798


In [4]:
df_mujeres = df[df['Genero']=='Mujer'].drop(columns=['Genero'])
df_hombres = df[df['Genero']=='Hombre'].drop(columns=['Genero'])
df.drop(columns='Genero')

,Peso,Altura
7620,61.235,162.402
1837,97.432,181.908
3311,73.324,172.459
9478,55.193,157.748
7805,56.886,151.798
...,...,...
6395,62.215,160.083
2714,83.840,177.866
3476,75.461,170.910
7084,65.461,166.137


In [5]:
p_mujer = len(df_mujeres)/len(df)
p_hombre = len(df_hombres)/len(df)
print(f'{p_mujer}+{p_hombre}={p_mujer+p_hombre}')

0.49975+0.50025=1.0


In [6]:
mu_p, mu_a = df.mean()
mu_p_m, mu_a_m = df_mujeres.mean()
mu_p_h, mu_a_h = df_hombres.mean()

In [7]:
def get_cov_mat_data(df):
    cov_mat = df.cov().to_numpy()
    std_x, std_y = np.sqrt(cov_mat.diagonal())
    std_xy = cov_mat[0][1]/std_x/std_y
    return std_x, std_y, std_xy
    
std_p, std_a, std_pa = get_cov_mat_data(df.drop(columns='Genero'))
std_p_m, std_a_m, std_pa_m = get_cov_mat_data(df_mujeres)
std_p_h, std_a_h, std_pa_h = get_cov_mat_data(df_hombres)

d = pd.DataFrame([['Todos', mu_p, std_p, mu_a, std_a, std_pa],
              ['Mujeres', mu_p_m, std_p_m, mu_a_m, std_a_m, std_pa_m], 
              ['Hombres', mu_p_h, std_p_h, mu_a_h, std_a_h, std_pa_h]],
    columns=['Grupo', 'Media peso', 'Desvio peso', 'Media altura', 'Desvio altura', 'Correlacion peso-altura'])

d

,Grupo,Media peso,Desvio peso,Media altura,Desvio altura,Correlacion peso-altura
0,Todos,72.802476,14.624699,168.113424,9.829018,0.924378
1,Mujeres,61.137895,8.671726,161.304713,6.897148,0.848971
2,Hombres,84.455398,8.983019,174.915330,7.281646,0.860449


In [8]:
n_bins = 40
hist, pedges, aedges = np.histogram2d(df['Peso'], df['Altura'], bins=n_bins)
hist_m, _, _ = np.histogram2d(df_mujeres['Peso'], df_mujeres['Altura'], bins=(pedges, aedges))
hist_h, _, _ = np.histogram2d(df_hombres['Peso'], df_hombres['Altura'], bins=(pedges, aedges))

hist /= len(df)
hist_m /= len(df_mujeres)
hist_h /= len(df_hombres)

In [9]:
aciertos = 0

def get_bin(a, x):
    ind = bisect.bisect(a, x)
    if ind == len(a):
        ind -= 1
    return ind
    
    
for _, persona in df_test.iterrows():
    p_pa = hist[get_bin(pedges, persona['Peso'])][get_bin(aedges, persona['Altura'])]
    p_pa_h = hist_h[get_bin(pedges_h, persona['Peso'])][get_bin(aedges_h, persona['Altura'])]

#print(p_pa)
    if p_pa != 0:
        p_h = p_pa_h * p_hombre / p_pa
    else:
        p_h = np.random.rand()
        
    guess = 'Hombre' if p_h > 0.5 else 'Mujer'
    #print(f"Adivine {'hombre' if p_h > 0.5 else 'mujer'} con p={p_h}")
    #print(f"Era {persona['Genero']}")
    if persona['Genero'] == guess:
        aciertos +=1
        
print(aciertos/len(df_test))


NameError: name 'pedges_h' is not defined

In [ ]:
def get_2dgaussian_dist(df):
    dist = stats.multivariate_normal(mean=df.mean(), cov=df.cov())
    return dist
    
def func():
    pa, pa_m, pa_h = (get_2dgaussian_dist(dataframe) for dataframe in [df, df_mujeres, df_hombres])

    pesos, alturas = np.mgrid[pedges[0]:pedges[-1]:1, aedges[0]:aedges[-1]:1]
    # alturas = np.linspace(min(df['Altura']), max(df['Altura']), 30)
    xy = np.dstack((pesos, alturas))

    figsize=(10, 5)
    fig1, (ax11, ax12) = plt.subplots(1, 2, figsize=figsize, 
                                      gridspec_kw={'width_ratios': [1, 1]})
    ax11.set_title('Todos')


    fig2, (ax21, ax22) = plt.subplots(1, 2, figsize=figsize, 
                                      gridspec_kw={'width_ratios': [1, 1]})
    ax21.set_title('Mujeres')

    fig2, (ax31, ax32) = plt.subplots(1, 2, figsize=figsize, 
                                      gridspec_kw={'width_ratios': [1, 1]})
    ax31.set_title('Hombres')

    ax11.imshow(hist, origin='lower', extent=(pedges[0], pedges[-1], aedges[0], aedges[-1]))
    ax21.imshow(hist_m, origin='lower', extent=(pedges[0], pedges[-1], aedges[0], aedges[-1]))
    ax31.imshow(hist_h, origin='lower', extent=(pedges[0], pedges[-1], aedges[0], aedges[-1]))

    ax12.contourf(pesos, alturas, pa.pdf(xy))
    ax22.contourf(pesos, alturas, pa_m.pdf(xy))
    ax32.contourf(pesos, alturas, pa_h.pdf(xy))

    ax11.axis('scaled')
    ax12.axis('scaled')
    ax21.axis('scaled')
    ax22.axis('scaled')
    ax31.axis('scaled')
    ax32.axis('scaled')

    plt.show()
    
func()

In [ ]:
def get_prob(dist, x, bin_size):
    return dist.cdf(x+bin_size/2)-dist.cdf(x-bin_size/2)

In [ ]:
bin_size = np.array([1, 0.5]) # 1kg y 0.5cm

aciertos = 0
for _, persona in df_test.iterrows():
    measures = np.array([persona['Peso'], persona['Altura']])
    p_pa = get_prob(pa, measures, bin_size)
    p_pa_h = get_prob(pa_h, measures, bin_size)
    
    if p_pa != 0:
        p_h = p_pa_h * p_hombre / p_pa
    else:
        p_h = 1 if persona['Altura'] >= mu_a else 0
    
    guess = 'Hombre' if p_h>0.5 else 'Mujer'
    if persona['Genero'] == guess:
        aciertos +=1
        
print(aciertos/len(df_test))